In [17]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random
import eatpy
import copy
from scipy.stats import qmc

In [16]:
from define_parameter_range import parameter_range,parameter_path

name_para = [row[0] for row in parameter_range]
l_bounds = [row[1] for row in parameter_range]
u_bounds = [row[2] for row in parameter_range]
n_decimals = [row[3] for row in parameter_range]

In [81]:
def generate_fabm_yaml_LHS(N):
    D = len(name_para)     # parameter size
    sampler = qmc.LatinHypercube(d=D)
    sample = sampler.random(n=N)
    sample_scaled = qmc.scale(sample, l_bounds, u_bounds)
    sample_rounded = np.zeros_like(sample_scaled)
    for i in range(np.shape(sample_scaled)[1]):
        sample_rounded[:,i] = np.round(sample_scaled[:,i],decimals=n_decimals[i])
        
    with eatpy.models.gotm.YAMLEnsemble("gotm.yaml", N) as gotm, eatpy.models.gotm.YAMLEnsemble("fabm.yaml", N) as fabm:
    # Commenting below will generate identical gotm_####.yaml files (do this if you want identical physics).
    #    gotm["surface/u10/scale_factor"] = np.random.lognormal(sigma=0.1, size=N)
    #    gotm["surface/v10/scale_factor"] = np.random.lognormal(sigma=0.1, size=N)
        # Link each gotm_####.yaml to the corresponding fabm_####.yaml
        gotm["fabm/yaml_file"] = fabm.file_paths
        # Set the MEMG model parameters
        for i in range(len(name_para)):
            fabm[parameter_path+name_para[i]] = sample_rounded[:,i]

    # Save the parameter values
    header = ','.join(name_para)
    np.savetxt("parameter-values-" + str(N) + ".csv",
               sample_rounded,
               delimiter=",",
               header=header,
               comments='')

### Initial population generation

In [82]:
maxgen = 10 # maximum number of generations/iterations allowed
N = 10 # population/ensemble size
tarfit = 0.99 # target fitness score
prob_mutation = 0.1 # probability of mutation for each parameter offspring during crossover
generate_fabm_yaml_LHS(N) # generate the initial population using Latin Hypercubic Sampling

# list of file names for the population
files = ['result_%04i.nc' % (i + 1) for i in range(N)]
files = np.array(files)

idx_time = range(100,366,10) # every 10 days after 100 days
idx_dep = range(1,110,10) # very 10 m from 1 m
tarvars = ["1p1z_no3"]#,"1p1z_chl"] # target variables

arr_fit = np.zeros((N,maxgen)) # array containing fitness

ds_tar = xr.open_dataset('result.nc') # control simulation results


INFO:root:Using template fabm.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR: 0.0

### fitness calculation
To do:

- consider rounding for disregarding subtle variations for correlations
- modify to accommodate the case when tarvars > 1

In [97]:
for igen in range(maxgen): # loop over the generation/iteration
    # Ensemble simulation
    !bash ensemble_simulation.sh

    # Fitness calculation
    for i in range(N): # loop over the population
        ds_ind = xr.open_dataset(files[i])
        for j in range(len(tarvars)):
            arr_fit[i,igen] = xr.corr(ds_tar[tarvars[j]].isel(time=idx_time,z=idx_dep),ds_ind[tarvars[j]].isel(time=idx_time,z=idx_dep)).values
    sorted_idx = np.argsort(arr_fit[:,igen]) # indices of the fitness array in the descending order (best fitness first, worst last)
    
    # elitism (keep the best individul for the next generation)
    source_file = files[sorted_idx][0]
    dest_file = "result_best.nc"
    !cp {source_file} {dest_file}
    
    # discard the worst individual for the next generation
    files_minus = files[sorted_idx[:-1]]

    # check whether to continue to the next generation
    if arr_fit.max() > tarfit: #satisfy the target?
        print('The fitness satisfies the target')
        break
    #elif median(arr_fit[:,igen]
        #print('The fitness is saturated')
        #break
    # Subsequent population generation
    else:
###FIX THE I AND J FOR LOOP (INCONSISTENT WITH THE LOOP ABOVE)
        # crossover
        offspring = np.zeros((N,len(name_para))) # initialize 2d array of the offspring values
        for j in range(N): # generate N individuals by crossover (the remaining two are generated via elitism and random)
            father = random.choice(files_minus)
            # Randomly select a mother, ensuring it is different from the father
            mother = random.choice([m for m in files_minus if m != father])
            father_yaml = 'fabm_'+father[-7:-3]+'.yaml'
            mother_yaml = 'fabm_'+mother[-7:-3]+'.yaml'
            # Open the YAML file and load its contents
            with open(father_yaml, 'r') as file:
                father_par = yaml.safe_load(file)
            with open(mother_yaml, 'r') as file:
                mother_par = yaml.safe_load(file)
            for i in range(len(name_para)):
                father_val = copy.deepcopy(father_par)
                mother_val = copy.deepcopy(mother_par)
                for part in (parameter_path+name_para[i]).split('/'):
                    father_val = father_val[part]
                    mother_val = mother_val[part]
                # set the value of either parent
                offspring[j,i] = random.choice([father_val,mother_val])
                # mutation with a specified probability: randomly select a value between the two parents
                if random.uniform(0,1) < prob_mutation:
                    offspring[j,i] = round(random.uniform(min(father_val,mother_val),max(father_val,mother_val)),n_decimals[i])
        with eatpy.models.gotm.YAMLEnsemble("fabm_next.yaml", N) as fabm:
            for i in range(len(name_para)):
                fabm[parameter_path+name_para[i]] = offspring[:,i]
        
        # elitism: keep the best individult for the next generation
        source_yaml = 'fabm_'+source_file[-7:-3]+'.yaml'
        dest_yaml = 'fabm_next_0002.yaml' # 0002 is hard coded and reserved for elitism
        !cp {source_yaml} {dest_yaml}
        
        # random: for exploration of parameter space and prevention from local minima, generate a totally random individual
        !ln -s fabm.yaml fabm_next.yaml
        with eatpy.models.gotm.YAMLEnsemble("fabm_next.yaml", 1) as fabm: # 1 is hard coded to generate only one ensemble 0001 
            for i in range(len(name_para)):
                fabm[parameter_path+name_para[i]] = [round(random.uniform(l_bounds[i],u_bounds[i]),n_decimals[i])] #[] is essential even though the array size is 1 here.
        
        # replace the next generation population by the current generation population 
        for i in range(N):
            source_yaml = 'fabm_next_'+files[i][-7:-3]+'.yaml'
            dest_yaml = 'fabm_'+files[i][-7:-3]+'.yaml'
            !mv {source_yaml} {dest_yaml}
            

Academic tradition requires you to cite works you base your article on.
If you use programs that use GNU Parallel to process data for an article in a
scientific publication, please cite:

  Tange, O. (2025, June 22). GNU Parallel 20250622 ('Павутина').
  Zenodo. https://doi.org/10.5281/zenodo.15717374

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

More about funding GNU Parallel and the citation notice:
https://www.gnu.org/software/parallel/parallel_design.html#citation-notice

To silence this citation notice: run 'parallel --citation' once.

Come on: You have run parallel 41 times. Isn't it about time 
you run 'parallel --citation' once to silence the citation notice?

 Initializing light...
    model type: memg/bio_fennel/light
    initialization succeeded.
 Initializing 1p1z...
    model type: memg/bio_fennel/1p1z
    initialization succeeded.
 SKIPPING model instance 2p2z because 

INFO:root:Using template fabm_next.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR

ln: fabm_next.yaml: File exists


INFO:root:Using template fabm_next.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR

Academic tradition requires you to cite works you base your article on.
If you use programs that use GNU Parallel to process data for an article in a
scientific publication, please cite:

  Tange, O. (2025, June 22). GNU Parallel 20250622 ('Павутина').
  Zenodo. https://doi.org/10.5281/zenodo.15717374

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

More about funding GNU Parallel and the citation notice:
https://www.gnu.org/software/parallel/parallel_design.html#citation-notice

To silence this citation notice: run 'parallel --citation' once.

Come on: You have run parallel 42 times. Isn't it about time 
you run 'parallel --citation' once to silence the citation notice?

 Initializing light...
    model type: memg/bio_fennel/light
    initialization succeeded.
 Initializing 1p1z...
    model type: memg/bio_fennel/1p1z
    initialization succeeded.
 SKIPPING model instance 2p2z because 

...

INFO:root:Using template fabm_next.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR

Academic tradition requires you to cite works you base your article on.
If you use programs that use GNU Parallel to process data for an article in a
scientific publication, please cite:

  Tange, O. (2025, June 22). GNU Parallel 20250622 ('Павутина').
  Zenodo. https://doi.org/10.5281/zenodo.15717374

This helps funding further development; AND IT WON'T COST YOU A CENT.
If you pay 10000 EUR you should feel free to use GNU Parallel without citing.

More about funding GNU Parallel and the citation notice:
https://www.gnu.org/software/parallel/parallel_design.html#citation-notice

To silence this citation notice: run 'parallel --citation' once.

Come on: You have run parallel 50 times. Isn't it about time 
you run 'parallel --citation' once to silence the citation notice?

 Initializing light...
    model type: memg/bio_fennel/light
    initialization succeeded.
 Initializing 1p1z...
    model type: memg/bio_fennel/1p1z
    initialization succeeded.
 SKIPPING model instance 2p2z because 

INFO:root:Using template fabm_next.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR

ln: fabm_next.yaml: File exists


INFO:root:Using template fabm_next.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR

In [95]:
files[0][-7:-3]

'0001'

In [98]:
arr_fit

array([[ 0.71837586,  0.73963511,  0.77469015,  0.79056299,  0.73234159,
         0.69190782,  0.16241965,  0.77952725,  0.7102378 ,  0.76990849],
       [ 0.84070426,  0.16470785,  0.16470785,  0.09086816, -0.07075705,
        -0.07075705, -0.07075705, -0.07075705, -0.07075705, -0.07667571],
       [ 0.84136671,  0.65061498,  0.76155829,  0.88847744,  0.09200832,
         0.08024443,  0.29290286,  0.21407218,  0.06719828,  0.07853929],
       [ 0.77310193,  0.81782711,  0.74475813,  0.90281576,  0.65854853,
         0.77795607,  0.28691363,  0.4617711 , -0.07667571, -0.06122876],
       [ 0.75853318,  0.73443997,  0.87850398,  0.85329676,  0.8085444 ,
         0.2309788 ,  0.20920584, -0.06108236,  0.07970979, -0.0609354 ],
       [ 0.77728164,  0.81785291,  0.67074049,  0.86067301,  0.41360933,
         0.85630453,  0.81687468,  0.15000698,  0.03513538,  0.03387909],
       [ 0.79329073,  0.71504915,  0.86364061,  0.76436162,  0.8603726 ,
         0.65686119,  0.89846075,  0.08013953

### Subsequent population generation
- elitism
- random
- crossover

In [79]:
prob_mutation = 0.1 # probability of mutation for each parameter offspring during crossover

# crossover
offspring = np.zeros((N,len(name_para))) # initialize 2d array of the offspring values
for j in range(N): # generate N individuals by crossover (the remaining two are generated via elitism and random)
    father = random.choice(files_minus)
    # Randomly select a mother, ensuring it is different from the father
    mother = random.choice([m for m in files_minus if m != father])
    father_yaml = 'fabm_'+father[-7:-3]+'.yaml'
    mother_yaml = 'fabm_'+mother[-7:-3]+'.yaml'
    # Open the YAML file and load its contents
    with open(father_yaml, 'r') as file:
        father_par = yaml.safe_load(file)
    with open(mother_yaml, 'r') as file:
        mother_par = yaml.safe_load(file)
    for i in range(len(name_para)):
        father_val = copy.deepcopy(father_par)
        mother_val = copy.deepcopy(mother_par)
        for part in (parameter_path+name_para[i]).split('/'):
            father_val = father_val[part]
            mother_val = mother_val[part]
        # set the value of either parent
        offspring[j,i] = random.choice([father_val,mother_val])
        # mutation with a specified probability: randomly select a value between the two parents
        if random.uniform(0,1) < prob_mutation:
            offspring[j,i] = round(random.uniform(min(father_val,mother_val),max(father_val,mother_val)),n_decimals[i])
with eatpy.models.gotm.YAMLEnsemble("fabm_next.yaml", N) as fabm:
    for i in range(len(name_para)):
        fabm[parameter_path+name_para[i]] = offspring[:,i]


# elitism: keep the best individult for the next generation
source_yaml = 'fabm_'+source_file[-7:-3]+'.yaml'
dest_yaml = 'fabm_next_0002.yaml' # 0002 is hard coded and reserved for elitism
!cp {source_yaml} {dest_yaml}

# random: for exploration of parameter space and prevention from local minima, generate a totally random individual
!ln -s fabm.yaml fabm_next.yaml
with eatpy.models.gotm.YAMLEnsemble("fabm_next.yaml", 1) as fabm: # 1 is hard coded to generate only one ensemble 0001 
    for i in range(len(name_para)):
        fabm[parameter_path+name_para[i]] = [round(random.uniform(l_bounds[i],u_bounds[i]),n_decimals[i])] #[] is essential even though the array size is 1 here.


INFO:root:Using template fabm_next.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR

ln: fabm_next.yaml: File exists


INFO:root:Using template fabm_next.yaml...
INFO:root:  instances/1p1z/parameters/attCHL: 0.02486
INFO:root:  instances/1p1z/parameters/I_thNH4: 0.0095
INFO:root:  instances/1p1z/parameters/D_p5NH4: 0.1
INFO:root:  instances/1p1z/parameters/NitriR: 0.05
INFO:root:  instances/1p1z/parameters/K_NO3: 2.0
INFO:root:  instances/1p1z/parameters/K_NH4: 2.0
INFO:root:  instances/1p1z/parameters/Vp0: 0.69
INFO:root:  instances/1p1z/parameters/K_Phy: 0.5
INFO:root:  instances/1p1z/parameters/PhyIS: 0.025
INFO:root:  instances/1p1z/parameters/PhyMR: 0.072
INFO:root:  instances/1p1z/parameters/Chl2C_m: 0.0535
INFO:root:  instances/1p1z/parameters/ZooAE_N: 0.75
INFO:root:  instances/1p1z/parameters/ZooBM: 0.1
INFO:root:  instances/1p1z/parameters/ZooER: 0.1
INFO:root:  instances/1p1z/parameters/ZooGR: 0.75
INFO:root:  instances/1p1z/parameters/ZooMR: 0.025
INFO:root:  instances/1p1z/parameters/LDeRRN: 0.01
INFO:root:  instances/1p1z/parameters/SDeRRN: 0.03
INFO:root:  instances/1p1z/parameters/CoagR

In [78]:
offspring[:,i]

array([0.022, 0.014, 0.07 , 0.37 , 4.7  , 4.6  , 3.4  , 2.5  , 0.022,
       0.2  , 0.079, 0.3  , 0.11 , 0.1  , 2.6  , 0.11 , 0.155, 0.069,
       0.026, 0.63 , 8.5  , 1.47 ])

In [73]:
parts = (parameter_path+name_para[i]).split('/')

In [40]:
parameter_path+name_para[i]

'instances/1p1z/parameters/attCHL'

In [49]:
fabm['instances/1p1z/parameters/attCHL']

0.02486

In [64]:
for i in range(len(name_para)):
        fabm[parameter_path+name_para[i]] = [round(random.uniform(l_bounds[i],u_bounds[i]),n_decimals[i])]


In [57]:
round(random.uniform(l_bounds[i],u_bounds[i]),n_decimals[i])

0.022

In [ ]:
r